# PHP 2020

## 8. Laravel

### 8.1 Tutorial

Start database:

In [ ]:
! docker run --name=mysql --net=host --rm --env MYSQL_ROOT_PASSWORD=root123 --env MYSQL_DATABASE=test --env MYSQL_USER=test --env MYSQL_PASSWORD=test123 -d mysql/mysql-server:8.0

In [ ]:
! while ! timeout 1 bash -c "echo > /dev/tcp/localhost/3306" 2> /dev/null; do sleep 1; done; echo "Done.";

#### Basics

Create a project:

In [ ]:
! composer create-project laravel/laravel demo

Change working directory:

In [ ]:
%cd demo

Try running the application:

In [ ]:
! php artisan serve --port 8888

Stop the above cell.

Simulate clean checkot:

In [ ]:
! rm -rf vendor .env

Restore the application:

In [ ]:
! composer install

In [ ]:
! php artisan serve --port 8888

Stop the above cell.

Add ```.env``` file:

In [ ]:
! cp .env.example .env

The newly created file: [.env](../../../edit/08_laravel/01_tutorial/demo/.env)

In [ ]:
! php artisan serve --port 8888

Stop the above cell.

In [ ]:
! php artisan key:generate

In [ ]:
! php artisan serve --port 8888

The application should now run crrectly.

Stop the above cell.

#### Codeception

In [ ]:
! composer require \
    codeception/codeception \
    codeception/module-asserts \
    codeception/module-phpbrowser \
    codeception/module-db \
    --dev --no-suggest

In [ ]:
! vendor/bin/codecept bootstrap tmp_bootrstrap

In [ ]:
! mv tmp_bootrstrap/tests/ tests_codeception

In [ ]:
! mv tmp_bootrstrap/codeception.yml .

In [ ]:
! rm -rf tmp_bootrstrap/

Replace the configuration in [codeception.yml](../../../edit/08_laravel/01_tutorial/demo/codeception.yml) by:

```yml
paths:
    tests: tests_codeception
    output: tests_codeception/_output
    data: tests_codeception/_data
    support: tests_codeception/_support
    envs: tests_codeception/_envs
actor_suffix: Tester
extensions:
    enabled:
        - Codeception\Extension\RunFailed
```

Run the tests:

In [ ]:
! vendor/bin/codecept run

Add a test:

In [ ]:
! vendor/bin/codecept generate:cept acceptance Homepage

Add below code in [tests_codeception/acceptance/HomepageCept.php](../../../edit/08_laravel/01_tutorial/demo/tests_codeception/acceptance/HomepageCept.php):

```php
<?php

$I = new AcceptanceTester($scenario);
$I->wantTo('see Laravel links on homepage');

$I->amOnPage('/');

$I->seeInTitle('Laravel');

$I->seeLink("Documentation", "https://laravel.com/docs");
$I->seeLink("Laracasts", "https://laracasts.com");
$I->seeLink("Forge", "https://forge.laravel.com");
```

To fix the URL in configuration open the [tests_codeception/acceptance.suite.yml](../../../edit/08_laravel/01_tutorial/demo/tests_codeception/acceptance.suite.yml) and change:

```yml
modules:
    enabled:
        - PhpBrowser:
            url: http://localhost/myapp
```

to

```yml
modules:
    enabled:
        - PhpBrowser:
            url: http://localhost:8888
```

Run the tests:

In [ ]:
import subprocess
subprocess.Popen(['php', 'artisan', 'serve', '--port', '8888'])

In [ ]:
! vendor/bin/codecept run

In [ ]:
! killall php7.4

#### Database

Try migrating the databse:

In [ ]:
! php artisan migrate

Edit the [.env](../../../edit/08_laravel/01_tutorial/demo/.env) file, and replace:

```
DB_CONNECTION=mysql
DB_HOST=127.0.0.1
DB_PORT=3306
DB_DATABASE=laravel
DB_USERNAME=root
DB_PASSWORD=
```

by

```
DB_CONNECTION=mysql
DB_HOST=127.0.0.1
DB_PORT=3306
DB_DATABASE=test
DB_USERNAME=test
DB_PASSWORD=test123
```

In [ ]:
! php artisan migrate

#### Codeception + Database

To connect to the database from tests, open the [tests_codeception/acceptance.suite.yml](../../../edit/08_laravel/01_tutorial/demo/tests_codeception/acceptance.suite.yml) and add

```yml
modules:
    enabled:
        - Db:
            dsn: 'mysql:host=127.0.0.1;dbname=test'
            user: 'test'
            password: 'test123'
```

Edit code in [tests_codeception/acceptance/HomepageCept.php](../../../edit/08_laravel/01_tutorial/demo/tests_codeception/acceptance/HomepageCept.php) and add below code at the end:

```php
$I->dontSeeInDatabase('users');
```

In [ ]:
import subprocess
subprocess.Popen(['php', 'artisan', 'serve', '--port', '8888'])

In [ ]:
! vendor/bin/codecept run

In [ ]:
! killall php7.4

#### Database dump

Dump the contents of the ```test``` database to ```dump.sql``` file:

In [ ]:
# Based on: https://gist.github.com/spalladino/6d981f7b33f6e0afe6bb
! docker exec mysql mysqldump -u root --password=root123 test > tests_codeception/_data/dump.sql

Open and inspect the generated file [tests_codeception/_data/dump.sql](../../../edit/08_laravel/01_tutorial/demo/tests_codeception/_data/dump.sql).

Open the [tests_codeception/acceptance.suite.yml](../../../edit/08_laravel/01_tutorial/demo/tests_codeception/acceptance.suite.yml) and replace:

```yml
dsn: 'mysql:host=127.0.0.1;dbname=test'
user: 'test'
password: 'test123'
```

by:

```yml
dsn: 'mysql:host=127.0.0.1;dbname=test'
user: 'test'
password: 'test123'
populate: true
cleanup: true
dump: tests_codeception/_data/dump.sql
```

In [ ]:
import subprocess
subprocess.Popen(['php', 'artisan', 'serve', '--port', '8888'])

In [ ]:
! vendor/bin/codecept run

In [ ]:
! killall php7.4

#### Cleanup

Change working directory:

In [ ]:
%cd ..

Stop database:

In [ ]:
! docker container stop mysql

Cleanup:

In [ ]:
! git clean -fdx .